In [1]:
from openstack import connection, identity
import numpy as np
import copy
import re
import openpyxl
from openpyxl import *
import os
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors, Alignment
from openpyxl.cell import Cell
import configparser, logging, operator

In [2]:
def connection_to_cluster(project):
    conn = connection.Connection(auth_url="http://controller:35357/v3",
                                 project_name=project,
                                 project_domain_name="default",
                                 user_domain_name="default",
                                 username="admin",
                                 password="QIP@t4Ay")
    return conn

In [3]:
class OSmap(object):
    
    def __init__ (self, connection='', OS=True, with_flavors=True, with_volumes=True, hypervisor_details=False):
        'Инит функция'
        if OS == True:
            self.__conn = connection
            self.__sl = [i for i in self.__conn.compute.servers(all_tenants=True)]
            self.__hl = [i for i in self.__conn.compute.hypervisors()]
            
            if with_flavors == True:
                self.__fd = self.__get_fl_dict()
                self.dict_flavors = copy.deepcopy(self.__fd)
            
            if with_volumes == True:
                self.__vd = self.__get_vl_dict()
                self.dict_volumes = copy.deepcopy(self.__vd)
                
                
            self.dict_hypervisors = self.__get_hp_dict(details=hypervisor_details)
            self.dict_servers = self.__get_vm_dict(flavors=with_flavors)
            self.list_names  = self.__get_names()
    
    
    def get_vm_by_name(self, vmn=''):
        'Функция возвращает объект ВМ на основании имени'
        for i in self.__sl:
            if i.name == vmn:
                return i 
    
    def get_vm_by_id(self, vmn=''):
        'Функция возвращает объект ВМ на основании ID'
        for i in self.__sl:
            if i.id == vmn:
                return i 
    
        
    
    def __get_hp_dict(self, details=False):
        'Функция возвращает словарь гипервизоров'
        d = {}
        for i in self.__hl:
            if details == True:
                d[i.name]=self.__conn.compute.get_hypervisor(i.id).to_dict()
            else:
                d[i.name]=i.to_dict()
        return d   
           
    def __get_fl_dict(self):
        'Функция возвращает словарь флейворов'
        d = {}
        for i in set([i.flavor['id'] for i in self.__sl]):
            d[i]=self.__conn.compute.get_flavor(flavor=i).to_dict()
        return d
    
    def __get_vl_dict(self):
        'Функция возвращает словарь вольюмов'
        d = {}
        for i in [i for i in self.__conn.block_store.volumes(all_tenants=True)]:
            d[i.id]=i.to_dict()
        return d
    
    
    def __get_names (self):
        'Функция возвращает список имен ВМ'
        a = [i.name for i in self.__sl]
        return a
    
            
    def __get_vm_dict(self,flavors=True):
        'Возвращает словарь с описанием ВМ из OS + cpu & ram'
        d = {}
        for i in self.__sl:
            d[i.name] = self.get_vm_by_name(vmn=i.name).to_dict()
            if flavors == True:
                d[i.name]['cpu'] = self.__fd[i.flavor['id']]['vcpus']
                d[i.name]['ram'] = self.__fd[i.flavor['id']]['ram']
        return d
        

In [4]:
conn = connection_to_cluster('admin')

In [5]:
##%%time
#x = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False)

In [6]:
#%%time
#y = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False)

In [7]:
#%%time
#z = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True)

In [8]:
#%%time
#x1 = OSmap(connection_to_cluster('admin'), with_flavors=False, with_volumes=False, hypervisor_details=True)

In [9]:
#%%time
#y1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=False, hypervisor_details=True)

In [10]:
%%time
z1 = OSmap(connection_to_cluster('admin'), with_flavors=True, with_volumes=True, hypervisor_details=True)

CPU times: user 4.34 s, sys: 167 ms, total: 4.51 s
Wall time: 38.4 s


## Копируем данные из класса. Создаем словарь групп

In [11]:
## Типы распределений
## 1 - распределение по зонам доступности
## 2 - распределение по ВУ
## 3 - распределение по модулю 2 (четные на одной ЗД, нечетные на другой ЗД)
## 4 - распределение по модулю 3 (1,2,3 на разных ЗД)
## 5 - распределение - master-stanby на разных ЗД
## 6 - распределение - migr ВМ
## 7 - распределение ВМ на мигр нодах

In [12]:
dRealVm = copy.deepcopy(z1.dict_servers)

In [13]:
lRealVmName = copy.deepcopy(z1.list_names)

In [14]:
dRealHyper = copy.deepcopy(z1.dict_hypervisors)

In [15]:
master_slave_list = [\
                    ['pg-master-standby','pg-master'],\
                    ['pg-attribute-standby-books','pg-attribute-books'],\
                    ['pg-attribute-standby-borders','pg-attribute-borders'],\
                    ['pg-attribute-standby-right-registry-entities','pg-attribute-right-registry-entities'],\
                    ['pg-reservation-01-standby','pg-reservation-01'],\
                     ['pg-master-replica-standby-01','pg-master-replica-01']\
                    
                    ]
reserve_list = [\
               'cn-e0f6.egron.tnx',\
               'cn-dce8.egron.tnx',\
               'cn-e624.egron.tnx'\
              ]

In [16]:
test_machines = [i for i in lRealVmName if ('bragin' in i or 'lacp' in i or 'Free' in i or 'is_efem' in i or 'gerc' in i)]

In [17]:
master_slave_list
total_ms_list = [i for j in master_slave_list for i in j]

In [18]:
[i for i in master_slave_list if 'pg-master-standby' in i ][0]

['pg-master-standby', 'pg-master']

In [19]:

with open('./group_prefix', 'r') as f:
    lGroupPrefix = f.readlines()
    lGroupPrefix = [i.replace('\n','') for i in lGroupPrefix]
f.close()


#with open('./group_prefixes_via_types2', 'r') as f:
#    lGroupPrefixTypes = f.readlines()
#    lGroupPrefixTypes = [i.replace('\n','') for i in lGroupPrefixTypes]
#    lGroupPrefixTypes = [i.split(',') for i in lGroupPrefixTypes]
#f.close()


with open('./group_prefix_via_types2', 'r') as f:
    lGroupPrefixTypes = f.readlines()
    lGroupPrefixTypes = [i.replace('\n','') for i in lGroupPrefixTypes]
    lGroupPrefixTypes = [i.split(',') for i in lGroupPrefixTypes]
f.close()


with open('./z1', 'r') as f:
    lAZ1 = f.readlines()
    lAZ1 = [i.replace('\n','') for i in lAZ1]
f.close()


with open('./z2', 'r') as f:
    lAZ2 = f.readlines()
    lAZ2 = [i.replace('\n','') for i in lAZ2]
f.close()
    
with open('./z3', 'r') as f:
    lAZ3 = f.readlines()
    lAZ3 = [i.replace('\n','') for i in lAZ3]
f.close()

with open('./very_new_cn_all', 'r') as f:
    lCN = f.readlines()
    lCN = [i.replace('\n','') for i in lCN]
    lCN = [i.split(',') for i in lCN]
f.close()

with open('./very_new_cn_migr', 'r') as f:
    lMigr = f.readlines()
    lMigr = [i.replace('\n','') for i in lMigr]
    lMigr = [i.split(',') for i in lMigr]
    lMigr = [i[1] for i in lMigr]
f.close()

In [20]:

dGroups = {}
for lprefix in lGroupPrefixTypes:
    prefix_name=lprefix[0]
    prefix_type=int(lprefix[1])
    sTemp = re.escape(prefix_name) + r"-\d{2,}"
    #print(prefix)
    regexp = re.compile(sTemp, re.IGNORECASE)
    dGroups[prefix_name]={}
    dGroups[prefix_name]['type'] = prefix_type
    dGroups[prefix_name]['vms'] = []
    for vm in lRealVmName:
        if regexp.match(vm):
            #print(vm)
            dGroups[prefix_name]['vms'].append(vm)
    dGroups[prefix_name]['vm_count'] = len(dGroups[prefix_name]['vms'])

In [21]:
#[[i, dGroups[i]['vm_count']] for i in dGroups.keys() if dGroups[i]['type'] == 1]


In [22]:
def make_xls():
    file="my_big_table.xlsx"
    wb = Workbook ()
    try:
        os.remove(file)
    except OSError:
        pass
    ws = wb.active
    
    ws.title=('Сводка')
    dh=copy.deepcopy(dUseHyper)
    dv = copy.deepcopy(dUseVms)
    l = [dh[h]['free_ram']/dh[h]['total_ram']  for h in dh.keys() if h not in reserve_list]
    m = [dh[h]['free_cpu']/dh[h]['total_cpu']  for h in dh.keys() if h not in reserve_list]
    #m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]
    vml = sum([dh[h]['vm_count'] for h in dh.keys()])
    hpl = [i for i in dh.keys()]
    migrations = count_migrations(d=dv)
    #print("Всего машин:", vml)
    #print("Всего ВУ:", len(hpl))
    #print("Забитость ВУ по RAM")
    #print("Самый забитый :", 1-min(l))
    #print("Наименее забитый :", 1-max(l))
    #print("Средняя загруженность :",  1 - sum(l)/len(l))
    #print("Среднедратичное отклонение по ВУ :",  np.std(l))
    #print('')
    #print("Перерасход vCPU относительно CPU ВУ ")
    #print("Самый забитый :", 1-min(m))
    #print("Наименее забитый :", 1-max(m))
    #print("Средняя загруженность :", 1 - sum(m)/len(m))
    ws.cell(column = 1, row = 2, value='Всего машин:')
    ws.cell(column = 1, row = 3, value='Всего ВУ')
    ws.cell(column = 1, row = 4, value='Самый забитый по RAM')
    ws.cell(column = 1, row = 5, value='Наименее забитый по RAM')
    ws.cell(column = 1, row = 6, value='Средняя загруженность ВУ по RAM')
    ws.cell(column = 1, row = 7, value='Среднеквадратичное отклонение по ВУ')
    ws.cell(column = 1, row = 8, value='')
    ws.cell(column = 1, row = 9, value='Самый забитый ВУ по CPU')
    ws.cell(column = 1, row = 10, value='Наименее забитый ВУ по CPU')
    ws.cell(column = 1, row = 11, value='Средняя загруженность по CPU')
    ws.cell(column = 1, row = 12, value='')
    ws.cell(column = 1, row = 13, value='Количество миграций')
    ws.cell(column = 2, row = 2, value=vml)
    ws.cell(column = 2, row = 3, value=len(hpl))
    ws.cell(column = 2, row = 4, value=1-min(l))
    ws.cell(column = 2, row = 5, value=1-max(l))
    ws.cell(column = 2, row = 6, value=1 - sum(l)/len(l))
    ws.cell(column = 2, row = 7, value=np.std(l))
    ws.cell(column = 2, row = 8, value='')
    ws.cell(column = 2, row = 9, value=1-min(m))
    ws.cell(column = 2, row = 10, value=1-max(m))
    ws.cell(column = 2, row = 11, value=1 - sum(m)/len(m))
    ws.cell(column = 2, row = 12, value='')
    ws.cell(column = 2, row = 13, value=migrations)

    ws = wb.create_sheet('Распределение ВМ по ВУ')
    warningFill = PatternFill(start_color='ffcc33',
                       end_color='ffcc33', fill_type='solid')
    criticalFill = PatternFill(start_color='ee4000',
                       end_color='ee4400', fill_type='solid')
    headersFill = PatternFill(start_color='ebecec',
                       end_color='ebecec', fill_type='solid')
    vm_headersFill = PatternFill(start_color='faebd7',
                       end_color='faebd7', fill_type='solid')
    top_headersFill = PatternFill(start_color='C2C8CC',
                       end_color='C2C8CC', fill_type='solid')

    row = 3
    col = 1
    t=dUseHyper
    ws.merge_cells('A1:E1')

    for i in t.keys():
        migration = t[i]['migr']
        if migration:
            mig = 'Да'
        else:
            mig = 'Нет'
        for rows in ws.iter_rows(min_row=row - 1, max_row=row - 1 , min_col=1, max_col=7):
            for cell in rows:
                cell.font = Font(italic=True)
                cell.alignment= Alignment(horizontal='center')
                cell.fill = top_headersFill
        ws.cell(column = col, row = row - 1 , value='Имя ВУ').alignment= Alignment(horizontal='left')
        ws.cell(column = col + 1, row = row  - 1, value='Номер зоны доступности')
        ws.cell(column = col + 2, row = row - 1, value='Переподписка по CPU')
        ws.cell(column = col + 3, row = row - 1, value='Свободная RAM')
        ws.cell(column = col + 4, row = row - 1, value='Количество ВМ на ВУ')
        ws.cell(column = col + 5, row = row - 1, value='Нода миграции')
        for rows in ws.iter_rows(min_row=row + 1, max_row=row + 1 , min_col=1, max_col=7):
            for cell in rows:
                cell.font = Font(italic=True) 
                cell.alignment= Alignment(horizontal='right')
        ws.cell(column=col, row=row).font = Font(name='Calibri', bold=True)
        for rows in ws.iter_rows(min_row=row , max_row=row, min_col=1, max_col=7):
            for cell in rows:
                  cell.fill = headersFill 
        ws.cell(column=col, row=row, value=i)
        ws.cell(column=col + 1, row=row, value=t[i]['az'])
        ws.cell(column=col + 2, row=row, value="{:10.2f}".format((t[i]['free_cpu']))).alignment = Alignment(horizontal='center')
        ws.cell(column=col + 3, row=row, value="{:10.2f}".format((t[i]['free_ram']))).alignment = Alignment(horizontal='center')
        ws.cell(column=col + 4, row=row, value=t[i]['vm_count'])
        ws.cell(column = col + 5, row = row , value=mig).alignment = Alignment(horizontal='center')
       # ws.cell(column=col + 5, row=row, value=t[i]['vm_critical']) 
       # ws.cell(column=col + 6, row=row, value=t[i]['vm_warning']) 
        row = row + 1
        for rows in ws.iter_rows(min_row=row, max_row=row, min_col=1, max_col=7):
            for cell in rows:
                cell.fill = vm_headersFill
                cell.alignment = Alignment(horizontal='center')
        ws.cell(column=col + 2, row=row, value='Имя ВМ').alignment = Alignment(horizontal='left')
        ws.cell(column=col + 3, row=row, value='vCPU')
        ws.cell(column=col + 4, row=row, value='RAM') 
        ws.cell(column=col + 5, row=row, value='Имя логической группы') 

        row = row + 1
        for j in t[i]['vms']:
            v = dUseVms[j]
            ws.cell(column=col + 2, row=row, value=v['name'])
            ws.cell(column=col + 3, row=row, value=v['cpu'])
            ws.cell(column=col + 4, row=row, value=v['ram']) 
            #if v['crit_rate'] == 2:
            #    crit = 1
            #elif v['crit_rate'] == 1:
            #    crit = 2
            #elif v['crit_rate'] == 0:
            #    crit = 'н/д'
            #ws.cell(column=col + 5, row=row, value=crit)
            #if v['crit_rate'] == 2:
            #    for rows in ws.iter_rows(min_row=row, max_row=row, min_col=3, max_col=7):
            #        for cell in rows:
            #              cell.fill = criticalFill
            #elif v['crit_rate'] == 1:
            #    for rows in ws.iter_rows(min_row=row, max_row=row, min_col=3, max_col=7):
            #        for cell in rows:
            #              cell.fill = warningFill

            ws.cell(column=col + 5, row=row, value=v['group']) 
            row = row + 1
        row = row + 2
    ws = wb.create_sheet('Информация по ВУ')
    row = 3 
    col = 1
    for az in [1,2,3]:
        aCnList = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az]
        #####
        ##### Надо переделать нижнюю часть и ссылаться на уже сделанный лист гиперов, а не дергать каждый раз словарь
        #####
        tCn  = len([i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az ])
        tFreeCpu = sum([dUseHyper[i]['free_cpu'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == az])
        tFreeRam = sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == az])
        tTotalCpu = sum([dUseHyper[i]['total_cpu'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == az])
        tTotalRam = sum([dUseHyper[i]['total_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == az])
        tPercFreeCpu = tFreeCpu/tTotalCpu
        tPercFreeRam = tFreeRam/tTotalRam
        tVm  = sum([dUseHyper[i]['vm_count'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == az])
        for rows in ws.iter_rows(min_row=row - 1, max_row=row - 1 , min_col=1, max_col=7):
            for cell in rows:
                cell.font = Font(italic=True)
                cell.alignment= Alignment(horizontal='center')
                cell.fill = top_headersFill
        for rows in ws.iter_rows(min_row=row, max_row=row , min_col=1, max_col=7):
            for cell in rows:
                cell.fill = headersFill
        ws.cell(column = col, row = row - 1 , value='Номер ЗД').alignment= Alignment(horizontal='left')
        ws.cell(column = col + 2, row = row - 1, value='Переподписка по CPU')
        ws.cell(column = col + 3, row = row - 1, value='Процент свободной RAM')
        ws.cell(column = col + 4, row = row - 1, value='Количество ВУ')
        ws.cell(column = col + 5, row = row - 1, value='Количество ВМ в зоне доступности')
        ws.cell(column = col, row = row , value=az).alignment= Alignment(horizontal='center')
        ws.cell(column = col + 2, row = row, value=tFreeCpu)
        ws.cell(column = col + 3, row = row, value=tPercFreeRam)
        ws.cell(column = col + 4, row = row, value=tCn)
        ws.cell(column = col + 5, row = row, value=tVm)
        row = row + 1
        for rows in ws.iter_rows(min_row=row, max_row=row, min_col=1, max_col=7):
            for cell in rows:
                cell.fill = vm_headersFill
                cell.alignment = Alignment(horizontal='center')
        ws.cell(column=col + 1, row=row, value='FQDN').alignment = Alignment(horizontal='left')       
        ws.cell(column=col + 2, row=row, value='Имя ВУ').alignment = Alignment(horizontal='left')
        ws.cell(column=col + 3, row=row, value='Переподписка по CPU')
        ws.cell(column=col + 4, row=row, value='Процент свободной RAM')
        ws.cell(column=col + 5, row=row, value='Количество свободной RAM')
        ws.cell(column=col + 6, row=row, value='Примечание') 
        for cn in aCnList:
            row = row + 1
            h = dUseHyper[cn]
            lName=h['label']
            lFreeCpu = h['free_cpu']
            lFreeRam = h['free_ram']
            lTotalCpu = h['total_cpu']
            lTotalRam = h['total_ram']
            lFreePercRam = lFreeRam/lTotalRam
            if h['migr']:
                lPrim='migr'
            else:
                lPrim=''
            ws.cell(column=col + 1, row=row, value=cn).alignment = Alignment(horizontal='left')
            ws.cell(column=col + 2, row=row, value=lName).alignment = Alignment(horizontal='left')
            ws.cell(column=col + 3, row=row, value=lFreeCpu)
            ws.cell(column=col + 4, row=row, value=lFreePercRam) 
            ws.cell(column=col + 5, row=row, value=lFreeRam) 
            ws.cell(column=col + 6, row=row, value=lPrim)
            #row = row + 1
        row = row + 3
    ws = wb.create_sheet('Список тестовых машин на удаление')
    row=2
    for rows in ws.iter_rows(min_row=row - 1, max_row=row - 1 , min_col=1, max_col=3):
        for cell in rows:
            cell.font = Font(italic=True)
            cell.alignment= Alignment(horizontal='center')
            cell.fill = top_headersFill
    ws.cell(column = 1, row = row - 1 , value='Имя ВМ').alignment= Alignment(horizontal='left')
    ws.cell(column = 2, row = row  - 1, value='RAM')
    ws.cell(column = 3, row = row - 1, value='vCPU')
    for v in test_machines:
        vn = dv[v]['name']
        vr = dv[v]['ram']
        vc = dv[v]['cpu']
        ws.cell(column=1, row=row, value=vn).alignment = Alignment(horizontal='left')
        ws.cell(column=2, row=row, value=vr).alignment = Alignment(horizontal='left')
        ws.cell(column=3, row=row, value=vc)
        row = row + 1
    wb.save(file)
    wb.close() 


In [24]:
#make_xls()
#dUseHyper

In [25]:
### основные функции по созданию словарей

def create_dict_computes_clean(z1,z2,z3,cn,migr):
    rd={}
    for i in cn:
        hname = i[0]
        hlabel = i[1]
        #print(hlabel,hname)
        if hlabel in z1:
            hzone=int('1')
        elif hlabel in z2:
            hzone=int('2')
        elif hlabel in z3:
            hzone=int('3')
       
        if hlabel in migr:
            hmigr=True
        else: 
            hmigr=False
        
        rd[hname]={}
        rd[hname]['migr']=hmigr
        rd[hname]['label']=hlabel
        rd[hname]['az']=hzone
        rd[hname]['total_cpu']=dRealHyper[hname]['vcpus']
        rd[hname]['total_ram']=dRealHyper[hname]['memory_size']
        rd[hname]['free_cpu']=dRealHyper[hname]['vcpus']
        rd[hname]['free_ram']=dRealHyper[hname]['memory_size']
        rd[hname]['vms']=[]
        rd[hname]['vm_count']=len(rd[hname]['vms'])
        
    return rd
        
def create_dict_vm_with_groups(vdict,gdict):
    rd={}
    for v in vdict.keys():
        rd[v]={}
        rd[v]['cpu']=vdict[v]['cpu']
        rd[v]['name']=vdict[v]['name']
        rd[v]['ram']=vdict[v]['ram']
        rd[v]['cn']=vdict[v]['hypervisor_hostname']
        rd[v]['cn_new'] = ''
        rd[v]['group']=''
        if v in total_ms_list:
            rd[v]['group'] = 'master_slave'
        for g in gdict.keys():
            if v in gdict[g]['vms']:
                rd[v]['group'] = g
                continue
            
    return rd

def create_dict_groups_via_az(gdict):
    rd={}
    rd=copy.deepcopy(gdict)
    for g in gdict:
        rd[g]['az']={}
        rd[g]['az'][1] = int('0')
        rd[g]['az'][2] = int('0')
        rd[g]['az'][3] = int('0')
        rd[g]['az_even'] = ''
        rd[g]['az_odd'] = ''
        rd[g]['hypers']={}
        for h in dVirtualHyper.keys():
            rd[g]['hypers'][h]=0

    
    return rd

In [26]:
##
## Функции распределения и поиска пула доступных гипервизоров
##



###### Необходимо сделать - описать дистрибуцию уровня 2 и 1, дистрибуцию для остальных машин (тип 0). - СДЕЛАНО


def distributuion_init(gdict,vdict,hdict,vms_on_migr):
    global dUseGroups, dUseVms, dUseHyper
    dUseGroups = copy.deepcopy(gdict)
    dUseVms    = copy.deepcopy(vdict)
    dUseHyper  = copy.deepcopy(hdict)
    dl=[]
    for v in dVirtualVm.keys():
        vmname = v
        vram = dUseVms[v]['ram'] 
        if vmname in test_machines:
            continue
        if   dVirtualVm[v]['group'] == 'master_slave':
            vmtype = int('5')
        elif dVirtualHyper[dVirtualVm[v]['cn']]['label'] in lMigr:
            vmtype = int('7')
        elif dVirtualVm[v]['group'] != '':
            vmtype = dVirtualGroups[dVirtualVm[v]['group']]['type']
        else:
            vmtype= int('0')
        dl.append([vmname,vmtype, vram])
    logger.debug(sorted(dl, key = operator.itemgetter(1,2), reverse=True))
    
    #distribution(vl=sorted(dl, key = lambda x: float(x[1]), reverse=True)) ## Первый - высокий тип
    distribution(vl=sorted(dl, key = operator.itemgetter(1,2), reverse=True)) ## Первыми - высокий тип и жирные
    #distribution(vl=sorted(dl, key = operator.itemgetter(1,2), reverse=False)) ## Первыми - низкий типа и легкие
    
    
def distribution(vl):
    for v in vl:
        vmname = v[0]
        vmtype = v[1]
        vm_cn  = dUseVms[vmname]['cn']
        cn_ram_after_distrib = dUseHyper[vm_cn]['free_ram'] - dUseVms[vmname]['ram']
        perc_cn = cn_ram_after_distrib / dUseHyper[vm_cn]['total_ram']
        hyper_list=[]
        ####
        #### Если ВМ на ВУ миграции - оставляем на ВУ 
        ####
        if vmname in vms_on_migr:
            #print( dUseVms[vmname]['cn'])
            hname = dUseVms[vmname]['cn']
            count_after_distrib(h=hname,vm=vmname)
            continue
        if vmtype == 1:
            hyper_list = az_distribution(vmname)
        
        elif vmtype == 2:
            hyper_list = cn_distribution(vmname)
        
        elif vmtype == 3:
            hyper_list = even_distribution(vmname)
            
        elif vmtype == 4: 
            hyper_list = triple_distribution(vmname)
            
        elif vmtype == 5:
            hyper_list = master_slave_distribution(vmname)
            
        elif vmtype == 6:
            hyper_list = migr_prefix_distribution(vmname)
            
        elif vmtype == 0:
            hyper_list = null_type_distribution(vmname)
        #print(vmname,vmtype)
        #print(vmname, set([dUseHyper[i]['az'] for i in hyper_list]))
        #print(vmname,vmtype)
        if vm_cn in hyper_list and perc_cn > 0.05:
            hyper_list=[vm_cn]
        main_distributuion_function(hl=hyper_list,vm=vmname)
        
        
def count_after_distrib(h,vm):
    az = dUseHyper[h]['az']
    vm_group = dUseVms[vm]['group']
    if vm_group != '' and vm_group != 'master_slave':
        dUseGroups[vm_group]['az'][az] = dUseGroups[vm_group]['az'][az] + 1
        dUseGroups[vm_group]['hypers'][h] = dUseGroups[vm_group]['hypers'][h] + 1
    vm_cpu = dUseVms[vm]['cpu']
    vm_ram = dUseVms[vm]['ram']
    dUseHyper[h]['free_ram'] = dUseHyper[h]['free_ram'] - vm_ram
    dUseHyper[h]['free_cpu'] = dUseHyper[h]['free_cpu'] - vm_cpu
    dUseHyper[h]['vms'].append(vm)
    dUseHyper[h]['vm_count']=len(dUseHyper[h]['vms'])
    dUseVms[vm]['cn_new']=h
    pass


def main_distributuion_function(hl, vm):
    #print(vm)
    top_list=[]
    vm_group = dUseVms[vm]['group']
    for h in hl:
        if (vm_group !='master_slave' and dUseHyper[h]['migr'] and dUseGroups[vm_group]['type'] != 6):
            #print('ouch! migration')
            continue
        if h in reserve_list:
            continue
        pram = dUseHyper[h]['free_ram']/dUseHyper[h]['total_ram']
        pcpu = dUseHyper[h]['free_cpu']/dUseHyper[h]['total_cpu']
        res_score = (0.00001 * pcpu + 1 * pram)
        top_list.append([res_score,h])
    #if vm == 'pg-attribute-18':
    #    print (top_list, hl)
    #print(top_list[:1])
    print(vm_group,vm)
    if (vm_group !='master_slave' and vm_group !=''
        and dUseGroups[vm_group]['type'] == 6 ):
        #logger.debug(vm)
        #logger.debug(top_list)
        print('OK')
    logger.debug(top_list)
    best_buddy = sorted(top_list, reverse=True)[:1][0][1]
   
    count_after_distrib(h=best_buddy,vm=vm)
    
    
def count_migrations(d):
    dv = copy.deepcopy(d)
    tl = [[i,dv[i]['cn'],dv[i]['cn_new']] for i in dv.keys() if (i not in test_machines)]
    otl = [[i,dv[i]['cn'],dv[i]['cn_new']] \
       for i in dv.keys() if (dUseVms[i]['cn_new'] ==  dv[i]['cn'] and i not in test_machines)]
    otl_vm = [i \
       for i in dv.keys() if (dUseVms[i]['cn_new'] ==  dv[i]['cn'] and i not in test_machines)]
    return len(tl) - len(otl)



In [27]:
#dUseHyper


In [28]:
#def keep_vm_migrate_on_its_host(vm):
#    rl = list(dUseVms[vm]['cn'])
#    return rl
#    pass

def migr_prefix_distribution(vm):
    rl = []
    #rl = [i for i in dUseHyper.keys() if dUseHyper[i]['migr'] and dUseHyper[i]['free_ram'] > 8192]
    for i in dUseHyper.keys():
        ram_left_after_ditribution = dUseHyper[i]['free_ram'] - dUseVms[vm]['ram']
        if dUseHyper[i]['migr'] and ram_left_after_ditribution > 8192:
            rl.append(i)
    if rl == []:
        rl = null_type_distribution(vm)
    return rl
    pass


def master_slave_distribution(vm):
    pair = [i for i in master_slave_list if vm in i ][0]
    #pair = [i for i in master_slave_list if vm in i ][0]
    pair_vm = [i for i in pair if i != vm][0]
    pair_cn = dUseVms[pair_vm]['cn_new']
    if pair_cn != '':
        rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] != dUseHyper[pair_cn]['az']]
    else:
        rl = [i for i in dUseHyper.keys()]
    return rl
    pass

def null_type_distribution(vm):
    if 'pg-attribute-standby' in vm:
        number = re.sub("\D", "", vm)
        vm_master = 'pg-attribute-' + number
        az_master = dUseHyper[dUseVms[vm_master]['cn_new']]['az']
        #print(vm,vm_master, az_master)
        rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] != az_master]
    else:
        logger.debug(vm)
#### Убери это в функцию !!! 
        alist=[]
        for a in [1,2,3]:
            memory_free=sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                    not dUseHyper[i]['migr']])
            memory_sum = sum([dUseHyper[i]['total_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                    not dUseHyper[i]['migr']])
            memory = memory_free / memory_sum
            alist.append([memory,a])
        logger.debug(alist)
        best_az = sorted(alist,reverse=True)[0][1]
        second_az = sorted(alist,reverse=True)[0][1]
        logger.debug(best_az)
        rl = [i for i in dUseHyper.keys() if not dUseHyper[i]['migr'] and dUseHyper[i]['az'] == best_az and\
             i not in reserve_list]
    logger.debug(rl)
    if rl == []:
        logger.debug('STOP')
    return rl
    pass

def az_distribution(vm):
    vm_group = dUseVms[vm]['group']
    azl = [dUseGroups[vm_group]['az'][i] for i in dUseGroups[vm_group]['az'].keys()]
    min_az = min(azl)
    min_az_list = [i for i in dUseGroups[vm_group]['az'].keys() if dUseGroups[vm_group]['az'][i] == min_az ]
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] in min_az_list]
    #print(vm,min_az,min_az_list)
    return rl
    
    pass

def cn_distribution(vm):
    vm_group = dUseVms[vm]['group']
    cnl = [dUseGroups[vm_group]['hypers'][i] for i in dUseGroups[vm_group]['hypers'].keys()\
          if not dUseHyper[i]['migr']]
    min_cnl = min(cnl)
    #print(min_cnl)
    ### УБЕРИ ЭТО БЛЯТЬ В ФУНКЦИЮ!!!!!! 
    '''
    alist=[]
    for a in [1,2,3]:
        memory_free=sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                not dUseHyper[i]['migr'] and i not in reserve_list])
        memory_sum = sum([dUseHyper[i]['total_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                not dUseHyper[i]['migr'] and i not in reserve_list])
        memory = memory_free / memory_sum
        alist.append([memory,a])
    logger.debug(alist)
    best_az = sorted(alist,reverse=True)[0][1]
    second_az = sorted(alist,reverse=True)[1][1]
    third_az = sorted(alist,reverse=True)[2][1]
    
    logger.debug(best_az)
        #rl = [i for i in dUseHyper.keys() if not dUseHyper[i]['migr'] and dUseHyper[i]['az'] == best_az]    
    rl = [i for i in dUseGroups[vm_group]['hypers'].keys() if not dUseHyper[i]['migr'] and\
          dUseGroups[vm_group]['hypers'][i] == min_cnl and i not in reserve_list and\
          dUseHyper[i]['az'] == best_az ]
    #print(rl)
    if rl == []:
        rl = [i for i in dUseGroups[vm_group]['hypers'].keys() if not dUseHyper[i]['migr'] and\
              dUseGroups[vm_group]['hypers'][i] == min_cnl and i not in reserve_list and\
          dUseHyper[i]['az'] == second_az] 
        #print('second',rl)
    if rl == []:
        rl = [i for i in dUseGroups[vm_group]['hypers'].keys() if not dUseHyper[i]['migr'] and\
              dUseGroups[vm_group]['hypers'][i] == min_cnl and i not in reserve_list and\
          dUseHyper[i]['az'] == third_az] 
        #print('third',rl)
    #print(rl)
    logger.debug(vm)
    logger.debug(rl)
    '''
    rl = [i for i in dUseGroups[vm_group]['hypers'].keys() if not dUseHyper[i]['migr'] and\
              dUseGroups[vm_group]['hypers'][i] == min_cnl and i not in reserve_list]
    if rl == []:
        rl = [i for i in dUseGroups[vm_group]['hypers'].keys() if not dUseHyper[i]['migr'] and\
              dUseGroups[vm_group]['hypers'][i] == min_cnl+1 and i not in reserve_list]    
    return rl
    pass



def even_distribution(vm):
    
    group = dUseVms[vm]['group']
    #ga1 = dUseGroups[group]['az1']
    #ga2 = dUseGroups[group]['az2']
    #ga3 = dUseGroups[group]['az3']
    if dUseGroups[group]['az_even'] == '':
        dUseGroups[group]['az_even'] = get_best_availability_zone(group=group)
    if dUseGroups[group]['az_odd'] == '':
        dUseGroups[group]['az_odd'] = get_best_availability_zone(group=group)
    odd_zone = dUseGroups[group]['az_odd']
    even_zone = dUseGroups[group]['az_even']
    stripped_name = vm.strip(group+'-')
    #print(group, )
    regexp = re.compile(r"\d\d-\d\d")
    if regexp.match(stripped_name):
        vmnumber_in_group = int(stripped_name.split('-')[1])
    else:
        vmnumber_in_group = int(stripped_name)
    #print(vm, vmnumber_in_group)
    if vmnumber_in_group % 2 == 0:
        az_to_land = even_zone
    else:
        az_to_land = odd_zone
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az_to_land]
    return rl





def triple_distribution(vm):
    #print(vm, 'TRIPLE')
    group = dUseVms[vm]['group']
    #ga1 = dUseGroups[group]['az1']
    #ga2 = dUseGroups[group]['az2']
    #ga3 = dUseGroups[group]['az3']
    vmnumber_in_group = int(vm.strip(group+'-'))
    az_to_land = (vmnumber_in_group % 3) + 1
    rl = [i for i in dUseHyper.keys() if dUseHyper[i]['az'] == az_to_land]
    return rl



def get_best_availability_zone(group):
    azl = [i for i in [1,2,3] if i not in [dUseGroups[group]['az_odd'], dUseGroups[group]['az_even']]]
    alist=[]
    for a in azl:
        memory=sum([dUseHyper[i]['free_ram'] for i in dUseHyper.keys() if dUseHyper[i]['az'] == a and\
                    not dUseHyper[i]['migr']])
        alist.append([memory,a])
    #print(sorted(alist, reverse=True))
    az = sorted(alist, reverse=True)[:1][0][1]
    #print(az)
    return az

In [29]:
def count_all(dh,dv,dg):
    l = [dh[h]['free_ram']/dh[h]['total_ram']  for h in dh.keys() if h not in reserve_list]
    m = [dh[h]['free_cpu']/dh[h]['total_cpu']  for h in dh.keys() if h not in reserve_list]
    #m = [hpd_res[h]['perc_cpu'] for h in hpd_res.keys()]
    vml = sum([dh[h]['vm_count'] for h in dh.keys()])
    hpl = [i for i in dh.keys()]
    #print(hpl)
    print("Всего машин:", vml)
    #print("Необходимо машинам vCPU :" , vm_res_total_list[0], ", RAM :", vm_res_total_list[1], "MB" )
    #print("На одном ВУ CPU :", cpu_per_cn, ", RAM :", ram_per_cn, "MB")
    print("Всего ВУ:", len(hpl))
    #print("Суммарно на ВУ :" , hyper_res_total_list[0], ", RAM :", hyper_res_total_list[1], "MB" )
    #print("Суммарная подписка по CPU :", over_total_list[0], ", RAM :", over_total_list[1])

    print("Забитость ВУ по RAM")
    print("Самый забитый :", 1-min(l))
    print("Наименее забитый :", 1-max(l))
    print("Средняя загруженность :",  1 - sum(l)/len(l))
    print("Среднедратичное отклонение по ВУ :",  np.std(l))
    print('')
    print("Перерасход vCPU относительно CPU ВУ ")
    print("Самый забитый :", 1-min(m))
    print("Наименее забитый :", 1-max(m))
    print("Средняя загруженность :", 1 - sum(m)/len(m))

In [30]:
#dUseHyper

In [31]:
#dRealHyper['cn-d720.egron.tnx']['memory_size']

In [32]:
###
### создаем виртуальную среду на основе базовых словарей
###
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

dVirtualHyper = create_dict_computes_clean(z1=lAZ1,z2=lAZ2,z3=lAZ3,migr=lMigr,cn=lCN)
dVirtualVm = create_dict_vm_with_groups(vdict=dRealVm,gdict=dGroups)
dVirtualGroups = create_dict_groups_via_az(gdict=dGroups)
ml = [i for i in dVirtualHyper.keys() if dVirtualHyper[i]['migr'] == True]
vms_on_migr = [i for j in ml for i in dVirtualVm.keys() if dVirtualVm[i]['cn'] == j]

In [33]:
#len([i for i in dVirtualVm.keys() if dVirtualVm[i]['group'] == 'master_slave'])

In [34]:
distributuion_init(gdict=dVirtualGroups,vdict=dVirtualVm,hdict=dVirtualHyper,vms_on_migr=vms_on_migr)

migr-gkn-oracle migr-gkn-oracle-24-err
OK
migr-filestore migr-filestore-02
OK
migr-log migr-log-01
OK
migr-grp-oracle migr-grp-oracle-24
OK
migr-gkn-oracle migr-gkn-oracle-24
OK
migr-arc migr-arc-67
OK
migr-arc migr-arc-64
OK
migr-arc migr-arc-60
OK
migr-arc migr-arc-57
OK
migr-arc migr-arc-46
OK
migr-arc migr-arc-40
OK
migr-arc migr-arc-36
OK
migr-arc migr-arc-34
OK
migr-arc migr-arc-31
OK
migr-node migr-node-140
OK
migr-node migr-node-131
OK
migr-node migr-node-112
OK
migr-node migr-node-110
OK
migr-node migr-node-106
OK
migr-node migr-node-105
OK
migr-node migr-node-104
OK
migr-node migr-node-076
OK
migr-gkn-oracle migr-gkn-oracle-61
OK
migr-arc migr-arc-85
OK
migr-arc migr-arc-80
OK
migr-arc migr-arc-81
OK
migr-arc migr-arc-77
OK
migr-arc migr-arc-82
OK
migr-gkn-oracle migr-gkn-oracle-85
OK
migr-gkn-oracle migr-gkn-oracle-82
OK
migr-gkn-oracle migr-gkn-oracle-81
OK
migr-gkn-oracle migr-gkn-oracle-50
OK
migr-arc migr-arc-74
OK
migr-arc migr-arc-43
OK
migr-gkn-oracle migr-gkn-oracle-

ppoz-db-proc-pkurp ppoz-db-proc-pkurp-06
ppoz-db-proc-pkurp ppoz-db-proc-pkurp-05
ppoz-db-proc-pkurp ppoz-db-proc-pkurp-04
ppoz-db-proc-pkurp ppoz-db-proc-pkurp-03
ppoz-db-proc-gmp ppoz-db-proc-gmp-06
ppoz-db-proc-gmp ppoz-db-proc-gmp-05
ppoz-db-proc-gmp ppoz-db-proc-gmp-04
ppoz-db-proc-gmp ppoz-db-proc-gmp-03
ppoz-db-proc-ppoz ppoz-db-proc-ppoz-02
ppoz-db-proc-ppoz ppoz-db-proc-ppoz-01
ppoz-db-proc-popd ppoz-db-proc-popd-02
ppoz-db-proc-popd ppoz-db-proc-popd-01
ppoz-db-proc-pkurp ppoz-db-proc-pkurp-02
ppoz-db-proc-pkurp ppoz-db-proc-pkurp-01
ppoz-db-proc-gmp ppoz-db-proc-gmp-02
ppoz-db-proc-gmp ppoz-db-proc-gmp-01
pmuf-db pmuf-db-01
pmuf-zabbix-db pmuf-zabbix-db-02
pmuf-zabbix-db pmuf-zabbix-db-01
ppoz-popd-process ppoz-popd-process-05
isurz-mq isurz-mq-06
isurz-mq isurz-mq-05
isurz-mq isurz-mq-04
isurz-mq isurz-mq-03
isurz-mq isurz-mq-02
isurz-mq isurz-mq-01
popd-bpd popd-bpd-06
popd-bpd popd-bpd-05
ppoz-bus-ext ppoz-bus-ext-04
ppoz-bus-ext ppoz-bus-ext-03
ppoz-bus-data ppoz-bus-dat

In [35]:
#[i for i in z1.dict_servers.keys() if 'pg-master-replica' in z1.dict_servers[i]['name']]

In [36]:
#dUseVms['pg-attribute-91'], dUseGroups['pg-attribute']

In [37]:
#len(z1.dict_servers.keys()), len(dUseVms)

In [38]:
#z1.dict_servers['pg-master-replica-standby-01']#, z1.dict_servers['pg-master-replica-standy-01']

In [39]:
count_all(dg=dUseGroups,dh=dUseHyper,dv=dUseVms)

Всего машин: 1653
Всего ВУ: 76
Забитость ВУ по RAM
Самый забитый : 0.982425191976634
Наименее забитый : 0.8160467320451072
Средняя загруженность : 0.8918649737037361
Среднедратичное отклонение по ВУ : 0.0605348845518

Перерасход vCPU относительно CPU ВУ 
Самый забитый : 3.7142857142857144
Наименее забитый : 2.142857142857143
Средняя загруженность : 2.901174168297456


In [40]:
make_xls()

/usr/lib/python3.6/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [ ]:
#dUseVms['pg-attribute-standby-02']

In [ ]:
tl = [[i,dUseVms[i]['cn'],dUseVms[i]['cn_new']] for i in dUseVms.keys()]
otl = [[i,dUseVms[i]['cn'],dUseVms[i]['cn_new']] \
       for i in dUseVms.keys() if dUseVms[i]['cn_new'] ==  dUseVms[i]['cn']]
otl_vm = [i \
       for i in dUseVms.keys() if dUseVms[i]['cn_new'] ==  dUseVms[i]['cn']]
migrations = len(tl) - len(otl)


In [ ]:
len(tl), len(otl)

In [ ]:
migrations = len(tl) - len(otl)
migrations

In [ ]:
dUseVms['iperf_lacp_test_client_1']

In [ ]:
#rm = [1,2,3,4,511,128,-1,-1,-1]
#min(rm)

In [ ]:
dUseGroups

In [ ]:
#dUseVms
len(master_slave_list)

In [ ]:
for c in range(0,len(master_slave_list)):
    print(c)
    pair = [i for i in master_slave_list][c]
    print(pair)
    vm1 = pair[0]
    vm2 = pair[1]
    cn1 = dUseVms[vm1]['cn_new']
    cn2 = dUseVms[vm2]['cn_new']
    az1 = dUseHyper[cn1]['az'] 
    az2 = dUseHyper[cn2]['az']

    print(vm1,vm2)
    print(cn1,cn2)
    print(az1,az2)

In [ ]:
tl = [[i,dUseVms[i]['cn'],dUseVms[i]['cn_new']] for i in dUseVms.keys()]

In [ ]:
otl = [[i,dUseVms[i]['cn'],dUseVms[i]['cn_new']] \
       for i in dUseVms.keys() if dUseVms[i]['cn_new'] ==  dUseVms[i]['cn']]

In [ ]:
otl_vm = [i \
       for i in dUseVms.keys() if dUseVms[i]['cn_new'] ==  dUseVms[i]['cn']]

In [ ]:
len(tl), len(otl)

In [ ]:
migrations = len(tl) - len(otl)

In [ ]:
migrations

In [ ]:
otl_vm

In [ ]:
otl

In [ ]:
sa = [i for i in dVirtualHyper.keys() if dVirtualHyper[i]['label'] in lMigr]

In [ ]:
sv = [i for i in dVirtualVm.keys() if dVirtualVm[i]['cn'] in sa]

In [ ]:
len(sv)

In [ ]:
sa

In [ ]:
sa_fail = ['cn-e0ba.egron.tnx', \
'cn-daa6.egron.tnx',\
'cn-db08.egron.tnx',\
'cn-e4bc.egron.tnx',\
'cn-e4ba.egron.tnx',\
'cn-e620.egron.tnx',\
'cn-da18.egron.tnx',\
'cn-da48.egron.tnx',\
'cn-da4a.egron.tnx',\
'cn-e826.egron.tnx',\
]

In [ ]:
len(sa), len(sa_fail)

In [ ]:
set(sa) - set(sa_fail)

In [ ]:
dVirtualHyper['cn-db26.egron.tnx']

In [ ]:
[i for i in dVirtualVm.keys() if dVirtualVm[i]['cn'] == 'cn-db26.egron.tnx']